In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
tf.keras.backend.set_floatx('float32')

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe=pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
train,test=train_test_split(dataframe,test_size=0.2)
train,val=train_test_split(train,test_size=0.2)

In [4]:
def df_to_dataset(dataframe,shuffle=True,batch_size=32):
    dataframe=dataframe.copy()
    labels=dataframe.pop('target')
    ds=tf.data.Dataset.from_tensor_slices((dict(dataframe),labels))
    if shuffle:
        ds=ds.shuffle(buffer_size=len(dataframe))
    ds=ds.batch(batch_size)
    return ds

In [5]:
train_d=df_to_dataset(train,batch_size=5)

In [6]:
example_batch=next(iter(train_d))[0]

In [7]:
def demo(feature_column):
    feature_layer=layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [8]:
age=feature_column.numeric_column('age')
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[44.]
 [37.]
 [57.]
 [46.]
 [34.]]


In [9]:
age_buchets=feature_column.bucketized_column(age,boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buchets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [10]:
thal=feature_column.categorical_column_with_vocabulary_list('thal',['fixed', 'normal', 'reversible'])
thal_one_hot=feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [26]:
thal_embedding=feature_column.embedding_column(thal,dimension=8)
demo(thal_embedding)

Instructions for updating:
Please use `layer.add_weight` method instead.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[[-0.13413231 -0.1200429   0.33735794  0.25106886 -0.02727805 -0.25419146
  -0.06174253 -0.5412365 ]
 [-0.13413231 -0.1200429   0.33735794  0.25106886 -0.02727805 -0.25419146
  -0.06174253 -0.5412365 ]
 [-0.13413231 -0.1200429   0.33735794  0.25106886 -0.02727805 -0.25419146
  -0.06174253 -0.5412365 ]
 [ 0.4125191   0.28769422  0.24232383 

In [ ]:
thal_hashed=feature_column.categorical_column_with_hash_bucket('thal',hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

In [17]:
trestbps=feature_column.numeric_column('trestbps')
trestbps_buckst=feature_column.bucketized_column(trestbps,boundaries=[130])

In [25]:
cross_feature=feature_column.crossed_column(['fbs','sex'],hash_bucket_size=100)
#demo(feature_column.indicator_column(cross_feature))

In [27]:
feature_columns=[]
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))
age_buchets=feature_column.bucketized_column(feature_column.numeric_column('age'),
                                            boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
thal=feature_column.categorical_column_with_vocabulary_list(
            'thal',['fixed', 'normal', 'reversible'])
thal_one_hot=feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)
thal_embedding=feature_column.embedding_column(thal,dimension=8)
feature_columns.append(thal_embedding)


In [28]:
feature_layer=tf.keras.layers.DenseFeatures(feature_columns)

In [29]:
batch_size=32
train_ds=df_to_dataset(train,batch_size=batch_size)
val_ds=df_to_dataset(val,shuffle=False,batch_size=batch_size)
test_ds=df_to_dataset(test,shuffle=False,batch_size=batch_size)

In [30]:
model=tf.keras.Sequential([
    feature_layer,
    layers.Dense(128,activation='relu'),
    layers.Dense(28,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'],
             run_eagerly=True)
model.fit(train_ds,
         validation_data=val_ds,
         epochs=5)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
7/7 [==============================] - 1s 75ms/step - loss: 3.4118 - accuracy: 0.5959 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
7/7 [==============================] - 0s 44ms/step - loss: 2.3278 - accuracy: 0.6839 - val_loss: 1.4923 - val_accuracy: 0.5306
Epoch 3/5
7/7 [==============================] - 0s 38ms/step - loss: 1.7472 - accuracy: 0.7513 - val_loss: 1.8227 - val_accuracy: 0.7551
Epoch 4/5
7/7 [==============================] - 0s 37ms/step - loss: 1.1455 - accurac